In [ ]:
import numpy as np
import pandas as pd
import json

########################SET folder to path of code directory in the project folder#####################
## SET PATH to code directory in the project folder
code_path = ""
import sys  
sys.path.insert(1, code_path)

########################SET folder to project directory path#####################
folder=""

from utils.utils import *
from utils.visualization import *

import matplotlib
import matplotlib.pyplot as plt

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

sensitive_attrs = ["CODE_GENDER","NAME_FAMILY_STATUS","AGE"]
fs = [0.5]

##
group_fair = ['DemographicParityRatio',
              'ConditionalDemographicDisparity',
              'EqualOpportunityDifference',
              'AverageOddsDifference',
              'PredictiveParity']
group_fair_codes = ['DPR','CDD','EOD','AOD','PPD']
indiv_fair = ['consistency_10','theil_index']

# One-off Feedback Integration and Retraining

## Labels ('fair'+'unfair')

In [ ]:
df_group_oneoff_ap1 = pd.read_csv(folder+'data/results/oneoff/Labels_fair+unfair/group_fairness_oneoff-Labels_Fair+Unfair.csv', delimiter=',')
df_indiv_oneoff_ap1 = pd.read_csv(folder+'data/results/oneoff/Labels_fair+unfair/individual_fairness_oneoff-Labels_Fair+Unfair.csv', delimiter=',')
df_acc_oneoff_ap1 = pd.read_csv(folder+'data/results/oneoff/Labels_fair+unfair/accuracy_oneoff-Labels_Fair+Unfair.csv', delimiter=',')

perc_change_df = get_percentage_change_oneoff(df_group_oneoff_ap1, group_fair, 
                                            df_indiv_oneoff_ap1, indiv_fair, 
                                             df_acc_oneoff_ap1,
                                            sensitive_attrs, fs)
##
perc_change_df.to_csv(folder+"data/results/oneoff/Labels_fair+unfair/perc_change_oneoff-Labels_Fair+Unfair.csv", index=False)
# perc_change_df

## Labels ('unfair') 

In [ ]:
df_group_oneoff_ap2 = pd.read_csv(folder+'data/results/oneoff/Labels_unfair/group_fairness_oneoff-Labels_Unfair.csv', delimiter=',')
df_indiv_oneoff_ap2 = pd.read_csv(folder+'data/results/oneoff/Labels_unfair/individual_fairness_oneoff-Labels_Unfair.csv', delimiter=',')
df_acc_oneoff_ap2 = pd.read_csv(folder+'data/results/oneoff/Labels_unfair/accuracy_oneoff-Labels_Unfair.csv', delimiter=',')

perc_change_df2 = get_percentage_change_oneoff(df_group_oneoff_ap2, group_fair, 
                                            df_indiv_oneoff_ap2, indiv_fair, 
                                             df_acc_oneoff_ap2,
                                            sensitive_attrs, fs)
##
perc_change_df2.to_csv(folder+"data/results/oneoff/Labels_unfair/perc_change_oneoff-Labels_Unfair.csv", index=False)
# perc_change_df2

## Labels+Weights ('fair'+'unfair')

In [ ]:
df_group_oneoff_ap3 = pd.read_csv(folder+'data/results/oneoff/Labels+Weights_fair+unfair/group_fairness_oneoff-Labels+Weights_Fair+Unfair.csv', delimiter=',')
df_indiv_oneoff_ap3 = pd.read_csv(folder+'data/results/oneoff/Labels+Weights_fair+unfair/individual_fairness_oneoff-Labels+Weights_Fair+Unfair.csv', delimiter=',')
df_acc_oneoff_ap3 = pd.read_csv(folder+'data/results/oneoff/Labels+Weights_fair+unfair/accuracy_oneoff-Labels+Weights_Fair+Unfair.csv', delimiter=',')

perc_change_df = get_percentage_change_oneoff(df_group_oneoff_ap3, group_fair, 
                                            df_indiv_oneoff_ap3, indiv_fair, 
                                             df_acc_oneoff_ap3,
                                            sensitive_attrs, fs)
##
perc_change_df.to_csv(folder+"data/results/oneoff/Labels+Weights_fair+unfair/perc_change_oneoff-Labels+Weights_Fair+Unfair.csv", index=False)
# perc_change_df

# IML-Labels

## 'fair'+'unfair'

In [ ]:
df_group = pd.read_csv(folder+'data/results/IML-Labels/fair+unfair/group_fairness_iml-labels_fair+unfair_with_cma.csv', delimiter=',')
df_indiv = pd.read_csv(folder+'data/results/IML-Labels/fair+unfair/individual_fairness_iml-labels_fair+unfair_with_cma.csv', delimiter=',')
df_acc = pd.read_csv(folder+'data/results/IML-Labels/fair+unfair/accuracy_iml-labels_fair+unfair_with_cma.csv', delimiter=',')

perc_change_df,cma_perc_change_df = get_percentage_change_IML(df_group, group_fair, 
                                                                    df_indiv, indiv_fair, 
                                                                    sensitive_attrs, fs)
perc_change_df.to_csv(folder+"data/results/IML-Labels/fair+unfair/perc_change_IML-Labels_Fair+Unfair_raw.csv", index=False)
cma_perc_change_df.to_csv(folder+"data/results/IML-Labels/fair+unfair/perc_change_IML-Labels_Fair+Unfair_cma.csv", index=False)

p_ids = perc_change_df['participant_id'].tolist()

### Cluster participants based on perc. change of group fairness measured at last iteration of CMA

In [ ]:
## df to array - keep only group fairness for clustering
perc_change_array = []
for i in perc_change_df.index:
    perc_change_array.append(perc_change_df.loc[i].tolist()[:-3])
cma_perc_change_array = []
for i in cma_perc_change_df.index:
    cma_perc_change_array.append(cma_perc_change_df.loc[i].tolist()[:-3])

##
clustered_data = cma_perc_change_array
metric = 'mean'

In [ ]:
## Search for optimal number of clusters
# candidate values for our number of cluster
parameters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 35, 40, 45]
silhouette_scores, best_score, best_grid = k_means_optimize_parameter(clustered_data, parameters, metric)
plot_silhouette_scores(silhouette_scores, parameters)

In [ ]:
nc = 3
cluster_ids, cluster_centroids, array_of_vectors_pca = k_means_pca(clustered_data, nc, metric, n_components=2)
colors = ['r','g','b','orange','purple', 'y','cyan','magenta','tab:blue']
plot_clusters(array_of_vectors_pca, cluster_ids, colors)
##
cluster_p_id_dict = {}
for i, cl_id in enumerate(cluster_ids):
    color = 'cluster_'+str(cl_id)
    if color not in cluster_p_id_dict:
        cluster_p_id_dict[color] = []
    cluster_p_id_dict[color].append((i, p_ids[i]))
cluster_p_id_dict

In [ ]:
clustered_data_df = cma_perc_change_df.copy(deep=True)
clustered_data_df.drop(columns=['consistency_10','theil_index'],inplace=True)
clustered_data_df['cluster_id'] = cluster_ids
for i,centroid in enumerate(cluster_centroids):
    centroid = centroid.tolist()
    centroid.append('centroid')
    centroid.append(i)
    clustered_data_df.loc[58+i] = centroid
clustered_data_df.to_csv(folder+"data/results/IML-Labels/fair+unfair/clustering/perc_change_IML-Labels_Unfair_clustering_cma_group.csv", index=False)

In [ ]:
for cl in cluster_p_id_dict:
    print(cl,len(cluster_p_id_dict[cl]))

#### Line plots per cluster

In [ ]:
for cl in cluster_p_id_dict:
    title = 'IML-Labels_Fair+Unfair_Clustering_CMA_Group_{}'.format(cl)
    fPath = folder+'data/results/IML-Labels/fair+unfair/clustering/'
    filename = 'lineGraphs_IML-Labels_Fair+Unfair_Clustering_CMA_Group_{}'.format(cl)
    image_type = '.jpeg'
    colors = ['r','g','b','orange','purple', 'y','cyan','magenta','tab:blue']
    cl_participants = [p_id[1] for p_id in cluster_p_id_dict[cl]]
    cl_participants.append(np.nan)
    print(cl, cl_participants)
    joint_plot_all_participants(title, 
                                fPath, filename, image_type, 
                                sensitive_attrs, 
                                group_fair, group_fair_codes, 
                                indiv_fair, 
                                fs, 
                                df_group.loc[df_group['participant_id'].isin(cl_participants)], 
                                df_indiv.loc[df_indiv['participant_id'].isin(cl_participants)], 
                                df_acc.loc[df_acc['participant_id'].isin(cl_participants)], 
                                colors, True, True)

#### Perc. change graphs per cluster

In [ ]:
## plot all fairness metrics
perc_ch_df = cma_perc_change_df
cluster_df = clustered_data_df
title = 'IML-Labels_Fair+Unfair Cluster {}'
file_name = 'percChange_Graphs_IML-Labels_Fair+Unfair_Clustering_CMA_Group_cl{}.pdf'
fPath = folder+'data/results/IML-Labels/fair+unfair/clustering/'
##
attrs  = ['CODE_GENDER','NAME_FAMILY_STATUS','AGE']
attrs_codes = ['Gender','Marit. Stat.','Age']
group_f = ['DemographicParityRatio',
           'ConditionalDemographicDisparity',
           'EqualOpportunityDifference',
           'AverageOddsDifference',
           'PredictiveParity']
group_fair_cod = ['DPR (+)','CDD (-)','EOD (+)','AOD (+)','PPD (+)']

perc_change_plots_per_cluster(perc_ch_df, cluster_df, title, file_name, fPath, attrs, attrs_codes, group_f, group_fair_cod)

### Perc. change plot of all participants

In [ ]:
## plot perc. change plot for all fairness metrics
perc_ch_df = cma_perc_change_df
title = 'IML-Labels_Fair+Unfair'
file_name = 'percChange_Graphs_IML-Labels_Fair+Unfair.pdf'
fPath = folder+'data/results/IML-Labels/fair+unfair/'
attrs  = ['CODE_GENDER','NAME_FAMILY_STATUS','AGE']
attrs_codes = ['Gender','Marit. Stat.','Age']
group_f = ['DemographicParityRatio','ConditionalDemographicDisparity','EqualOpportunityDifference',
                    'AverageOddsDifference','PredictiveParity','indiv.']
group_fair_cod = ['DPR (+)','CDD (-)','EOD (+)','AOD (+)','PPD (+)']

perc_change_plots(perc_ch_df, title, file_name, fPath, attrs, attrs_codes, group_f, group_fair_cod)

### Line (Raw and CMA) plots

#### Joint plots for all participants

In [ ]:
title = 'IML-Labels_Fair+Unfair_all'
fPath = folder+'data/results/IML-Labels/fair+unfair/'
image_type = '.jpeg'
filename = 'all_participants'
joint_plot_all_participants(title, fPath, filename, image_type, 
                            sensitive_attrs, 
                            group_fair, group_fair_codes, 
                            indiv_fair, 
                            fs, 
                            df_group, df_indiv, df_acc, 
                            colors, True, True)

#### Plots per participant

In [ ]:
title = 'IML-Labels_Fair+Unfair_{}'
fPath = folder+'data/results/IML-Labels/fair+unfair/'
image_type = '.jpeg'
plots_per_participant(title, fPath, image_type, 
                      sensitive_attrs, 
                      group_fair, group_fair_codes, 
                      indiv_fair, 
                      fs, 
                      df_group, df_indiv, df_acc, 
                      colors, True)

## 'unfair'

In [ ]:
df_group = pd.read_csv(folder+'data/results/IML-Labels/unfair/group_fairness_iml-labels_unfair_with_cma.csv', delimiter=',')
df_indiv = pd.read_csv(folder+'data/results/IML-Labels/unfair/individual_fairness_iml-labels_unfair_with_cma.csv', delimiter=',')
df_acc = pd.read_csv(folder+'data/results/IML-Labels/unfair/accuracy_iml-labels_unfair_with_cma.csv', delimiter=',')

perc_change_df, cma_perc_change_df = get_percentage_change_IML(df_group, group_fair, 
                                                               df_indiv, indiv_fair, 
                                                               sensitive_attrs, fs)
perc_change_df.to_csv(folder+"data/results/IML-Labels/unfair/perc_change_IML-Labels_Unfair_raw.csv", index=False)
cma_perc_change_df.to_csv(folder+"data/results/IML-Labels/unfair/perc_change_IML-Labels_Unfair_cma.csv", index=False)

p_ids = perc_change_df['participant_id'].tolist()

### Cluster participants based on perc. change of group fairness measured at last iteration of CMA

In [ ]:
## df to array - keep only group fairness for clustering
perc_change_array = []
for i in perc_change_df.index:
    perc_change_array.append(perc_change_df.loc[i].tolist()[:-3])
cma_perc_change_array = []
for i in cma_perc_change_df.index:
    cma_perc_change_array.append(cma_perc_change_df.loc[i].tolist()[:-3])

##
clustered_data = cma_perc_change_array
metric = 'mean'

In [ ]:
## Search for optimal number of clusters
# candidate values for our number of cluster
parameters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 35, 40, 45]
silhouette_scores, best_score, best_grid = k_means_optimize_parameter(clustered_data, parameters, metric)
plot_silhouette_scores(silhouette_scores, parameters)

In [ ]:
nc = 3
cluster_ids, cluster_centroids, array_of_vectors_pca = k_means_pca(clustered_data, nc, metric, n_components=2)
colors = ['r','g','b','orange','purple', 'y','cyan','magenta','tab:blue']
plot_clusters(array_of_vectors_pca, cluster_ids, colors)
##
cluster_p_id_dict = {}
for i, cl_id in enumerate(cluster_ids):
    color = 'cluster_'+str(cl_id)
    if color not in cluster_p_id_dict:
        cluster_p_id_dict[color] = []
    cluster_p_id_dict[color].append((i, p_ids[i]))
cluster_p_id_dict

In [ ]:
clustered_data_df = cma_perc_change_df.copy(deep=True)
clustered_data_df.drop(columns=['consistency_10','theil_index'],inplace=True)
clustered_data_df['cluster_id'] = cluster_ids
for i,centroid in enumerate(cluster_centroids):
    centroid = centroid.tolist()
    centroid.append('centroid')
    centroid.append(i)
    clustered_data_df.loc[58+i] = centroid
clustered_data_df.to_csv(folder+"data/results/IML-Labels/unfair/clustering/perc_change_IML-Labels_Unfair_clustering_cma_group.csv", index=False)
# clustered_data_df.to_csv("perc_change_IML-Labels_Unfair_clustering_cma_group+indiv.csv", index=False)

In [ ]:
for cl in cluster_p_id_dict:
    print(cl,len(cluster_p_id_dict[cl]))

#### Line plots per cluster

In [ ]:
for cl in cluster_p_id_dict:
    title = 'IML-Labels_Unfair_Clustering_CMA_Group_{}'.format(cl)
    fPath = folder+'data/results/IML-Labels/unfair/clustering/'
    filename = 'lineGraphs_IML-Labels_Unfair_Clustering_CMA_Group_{}'.format(cl)
    image_type = '.jpeg'
    colors = ['r','g','b','orange','purple', 'y','cyan','magenta','tab:blue']
    cl_participants = [p_id[1] for p_id in cluster_p_id_dict[cl]]
    cl_participants.append(np.nan)
    print(cl, cl_participants)
    joint_plot_all_participants(title, 
                                fPath, filename, image_type, 
                                sensitive_attrs, 
                                group_fair, group_fair_codes, 
                                indiv_fair, 
                                fs, 
                                df_group.loc[df_group['participant_id'].isin(cl_participants)], 
                                df_indiv.loc[df_indiv['participant_id'].isin(cl_participants)], 
                                df_acc.loc[df_acc['participant_id'].isin(cl_participants)], 
                                colors, True, True)

#### Perc. change graphs per cluster

In [ ]:
## plot all fairness metrics
perc_ch_df = cma_perc_change_df
cluster_df = clustered_data_df
title = 'IML-Labels_Unfair Cluster {}'
file_name = 'percChange_Graphs_IML-Labels_Unfair_Clustering_CMA_Group_cl{}.pdf'
fPath = folder+'data/results/IML-Labels/unfair/clustering/'
##
attrs  = ['CODE_GENDER','NAME_FAMILY_STATUS','AGE']
attrs_codes = ['Gender','Marit. Stat.','Age']
group_f = ['DemographicParityRatio',
           'ConditionalDemographicDisparity',
           'EqualOpportunityDifference',                    
           'AverageOddsDifference',
           'PredictiveParity']
group_fair_cod = ['DPR (+)','CDD (-)','EOD (+)','AOD (+)','PPD (+)']

perc_change_plots_per_cluster(perc_ch_df, cluster_df, title, file_name, fPath, attrs, attrs_codes, group_f, group_fair_cod)

In [ ]:
## plot only DPR and AOD
perc_ch_df = cma_perc_change_df
cluster_df = clustered_data_df
title = 'IML-Labels_Unfair Cluster {}'
file_name = 'percChange_Graphs_IML-Labels_Unfair_Clustering_CMA_Group_cl{}_short.pdf'
fPath = folder+'data/results/IML-Labels/unfair/clustering/'
##
attrs  = ['CODE_GENDER','NAME_FAMILY_STATUS','AGE']
attrs_codes = ['Gender','Marit. Stat.','Age']
group_f = ['DemographicParityRatio','AverageOddsDifference']
group_fair_cod = ['DPR (+)','AOD (+)']

perc_change_plots_per_cluster(perc_ch_df, cluster_df, title, file_name, fPath, attrs, attrs_codes, group_f, group_fair_cod)

### Perc. change plot of all participants

In [ ]:
## plot perc. change plot for all fairness metrics
perc_ch_df = cma_perc_change_df
title = 'IML-Labels_Unfair'
file_name = 'percChange_Graphs_IML-Labels_Unfair.pdf'
fPath = folder+'data/results/IML-Labels/unfair/'
attrs  = ['CODE_GENDER','NAME_FAMILY_STATUS','AGE']
attrs_codes = ['Gender','Marit. Stat.','Age']
group_f = ['DemographicParityRatio',
           'ConditionalDemographicDisparity',
           'EqualOpportunityDifference',
            'AverageOddsDifference',
           'PredictiveParity','indiv.']
group_fair_cod = ['DPR (+)','CDD (-)','EOD (+)','AOD (+)','PPD (+)']

perc_change_plots(perc_ch_df, title, file_name, fPath, attrs, attrs_codes, group_f, group_fair_cod)

In [ ]:
## plot perc. change plot for DPR and AOD
perc_ch_df = cma_perc_change_df
title = 'IML-Labels_Unfair'
file_name = 'percChange_Graphs_IML-Labels_Unfair_short.pdf'
fPath = folder+'data/results/IML-Labels/unfair/'
attrs  = ['CODE_GENDER','NAME_FAMILY_STATUS','AGE']
attrs_codes = ['Gender','Marit. Stat.','Age']
group_f = ['DemographicParityRatio','AverageOddsDifference']
group_fair_cod = ['DPR (+)','AOD (+)']

perc_change_plots(perc_ch_df, title, file_name, fPath, attrs, attrs_codes, group_f, group_fair_cod)

### Line (Raw and CMA) plots

#### Joint plots for all participants

In [ ]:
title = 'IML-Labels_Unfair_all'
fPath = folder+'data/results/IML-Labels/unfair/'
image_type = '.jpeg'
filename = 'all_participants'
joint_plot_all_participants(title, fPath, filename, image_type, 
                            sensitive_attrs, 
                            group_fair, group_fair_codes, 
                            indiv_fair, 
                            fs, 
                            df_group, df_indiv, df_acc, 
                            colors, True, True)

#### Plots per participant

In [ ]:
title = 'IML-Labels_Unfair_{}'
fPath = folder+'data/results/IML-Labels/unfair/'
image_type = '.jpeg'
plots_per_participant(title, fPath, image_type, 
                      sensitive_attrs, 
                      group_fair, group_fair_codes, 
                      indiv_fair, 
                      fs, 
                      df_group, df_indiv, df_acc, 
                      colors, True)

#### Line graphs for DPR and AOD and 3 protected attributes for participant 608aa18a3c2dfd212a1a77a9 (IML-Labels_Unfair)

In [ ]:
group_f = ['DemographicParityRatio','AverageOddsDifference']
group_fair_cod = ['DPR','AOD']
colors = ['r','g','b','orange','purple', 'y','cyan','magenta','tab:blue']
indiv_f = []
title = 'IML-Labels_Unfair_{}'
fPath = folder+'data/results/IML-Labels/unfair/'
image_type = '.pdf'
line_graphs_of_participant(title, fPath, image_type, 
                      sensitive_attrs, 
                      group_f, group_fair_cod, 
                      indiv_f, 
                      fs, 
                      df_group, df_indiv, df_acc, 
                      colors, True, '608aa18a3c2dfd212a1a77a9')

# IML-Labels+Weights

## 'fair'+'unfair'

In [ ]:
df_group = pd.read_csv(folder+'data/results/IML-Labels+Weights/fair+unfair/group_fairness_IML-Labels+Weights_Fair+Unfair_with_cma.csv', delimiter=',')
df_indiv = pd.read_csv(folder+'data/results/IML-Labels+Weights/fair+unfair/individual_fairness_IML-Labels+Weights_Fair+Unfair_with_cma.csv', delimiter=',')
df_acc = pd.read_csv(folder+'data/results/IML-Labels+Weights/fair+unfair/accuracy_IML-Labels+Weights_Fair+Unfair_with_cma.csv', delimiter=',')

perc_change_df, cma_perc_change_df = get_percentage_change_IML(df_group, group_fair, 
                                                               df_indiv, indiv_fair, 
                                                               sensitive_attrs, fs)
perc_change_df.to_csv(folder+"data/results/IML-Labels+Weights/fair+unfair/perc_change_IML-Labels+Weights_Fair+Unfair_raw.csv", index=False)
cma_perc_change_df.to_csv(folder+"data/results/IML-Labels+Weights/fair+unfair/perc_change_IML-Labels+Weights_Fair+Unfair_cma.csv", index=False)

p_ids = perc_change_df['participant_id'].tolist()

### Cluster participants based on perc. change of group fairness measured at last iteration of CMA

In [ ]:
## df to array - keep only group fairness for clustering
perc_change_array = []
for i in perc_change_df.index:
    perc_change_array.append(perc_change_df.loc[i].tolist()[:-3])
cma_perc_change_array = []
for i in cma_perc_change_df.index:
    cma_perc_change_array.append(cma_perc_change_df.loc[i].tolist()[:-3])

##
clustered_data = cma_perc_change_array
metric = 'mean'

In [ ]:
## Search for optimal number of clusters
# candidate values for our number of cluster
parameters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 35, 40, 45]
silhouette_scores, best_score, best_grid = k_means_optimize_parameter(clustered_data, parameters, metric)
plot_silhouette_scores(silhouette_scores, parameters)

In [ ]:
nc = 3
cluster_ids, cluster_centroids, array_of_vectors_pca = k_means_pca(clustered_data, nc, metric, n_components=2)
colors = ['r','g','b','orange','purple', 'y','cyan','magenta','tab:blue']
plot_clusters(array_of_vectors_pca, cluster_ids, colors)
##
cluster_p_id_dict = {}
for i, cl_id in enumerate(cluster_ids):
    color = 'cluster_'+str(cl_id)
    if color not in cluster_p_id_dict:
        cluster_p_id_dict[color] = []
    cluster_p_id_dict[color].append((i, p_ids[i]))
cluster_p_id_dict

In [ ]:
clustered_data_df = cma_perc_change_df.copy(deep=True)
clustered_data_df.drop(columns=['consistency_10','theil_index'],inplace=True)
clustered_data_df['cluster_id'] = cluster_ids
for i,centroid in enumerate(cluster_centroids):
    centroid = centroid.tolist()
    centroid.append('centroid')
    centroid.append(i)
    clustered_data_df.loc[58+i] = centroid
clustered_data_df.to_csv(folder+"data/results/IML-Labels+Weights/fair+unfair/clustering/perc_change_IML-Labels+Weights_Fair+Unfair_clustering_cma_group.csv", index=False)

In [ ]:
for cl in cluster_p_id_dict:
    print(cl,len(cluster_p_id_dict[cl]))

#### Line plots per cluster

In [ ]:
for cl in cluster_p_id_dict:
    title = 'IML-Labels+Weights_Fair+Unfair_CMA_{}'.format(cl)
    fPath = folder+'data/results/IML-Labels+Weights/fair+unfair/clustering/'
    filename = cl
    image_type = '.jpeg'
    cl_participants = [p_id[1] for p_id in cluster_p_id_dict[cl]]
    cl_participants.append(np.nan)
    print(cl, cl_participants)
    joint_plot_all_participants(title, 
                                fPath, filename, image_type, 
                                sensitive_attrs, 
                                group_fair, group_fair_codes, 
                                indiv_fair, 
                                fs, 
                                df_group.loc[df_group['participant_id'].isin(cl_participants)], 
                                df_indiv.loc[df_indiv['participant_id'].isin(cl_participants)], 
                                df_acc.loc[df_acc['participant_id'].isin(cl_participants)], 
                                colors, True, True)

#### Perc. change graphs per cluster

In [ ]:
## plot all fairness metrics
perc_ch_df = cma_perc_change_df
cluster_df = clustered_data_df
title = 'IML-Labels+Weights_Fair+Unfair Cluster {}'
file_name = 'percChange_Graphs_IML-Labels+Weights_Fair+Unfair_Clustering_CMA_Group_cl{}.pdf'
fPath = folder+'data/results/IML-Labels+Weights/fair+unfair/clustering/'
##
attrs  = ['CODE_GENDER','NAME_FAMILY_STATUS','AGE']
attrs_codes = ['Gender','Marit. Stat.','Age']
group_f = ['DemographicParityRatio','ConditionalDemographicDisparity','EqualOpportunityDifference',
                    'AverageOddsDifference','PredictiveParity']
group_fair_cod = ['DPR (+)','CDD (-)','EOD (+)','AOD (+)','PPD (+)']

perc_change_plots_per_cluster(perc_ch_df, cluster_df, title, file_name, fPath, attrs, attrs_codes, group_f, group_fair_cod)

### Perc. change plot of all participants

In [ ]:
## plot perc. change plot for all fairness metrics
perc_ch_df = cma_perc_change_df
title = 'IML-Labels+Weights_Fair+Unfair'
file_name = 'percChange_Graphs_IML-Labels+Weights_Fair+Unfair.pdf'
fPath = folder+'data/results/IML-Labels+Weights/fair+unfair/'
attrs  = ['CODE_GENDER','NAME_FAMILY_STATUS','AGE']
attrs_codes = ['Gender','Marit. Stat.','Age']
group_f = ['DemographicParityRatio','ConditionalDemographicDisparity','EqualOpportunityDifference',
                    'AverageOddsDifference','PredictiveParity','indiv.']
group_fair_cod = ['DPR (+)','CDD (-)','EOD (+)','AOD (+)','PPD (+)']

perc_change_plots(perc_ch_df, title, file_name, fPath, attrs, attrs_codes, group_f, group_fair_cod)

### Line (Raw and CMA) plots

#### Joint plots for all participants

In [ ]:
title = 'IML-Labels+Weights_Fair+Unfair_all'
fPath = folder+'data/results/IML-Labels+Weights/fair+unfair/'
image_type = '.jpeg'
filename = 'all_participants'
joint_plot_all_participants(title, fPath, filename, image_type, 
                            sensitive_attrs, 
                            group_fair, group_fair_codes, 
                            indiv_fair, 
                            fs, 
                            df_group, df_indiv, df_acc, 
                            colors, True, True)

#### Plots per participant

In [ ]:
title = 'IML-Labels+Weights_Fair+Unfair_{}'
fPath = folder+'data/results/IML-Labels+Weights/fair+unfair/'
image_type = '.jpeg'
plots_per_participant(title, fPath, image_type, 
                      sensitive_attrs, 
                      group_fair, group_fair_codes, 
                      indiv_fair, 
                      fs, 
                      df_group, df_indiv, df_acc, 
                      colors, True)

# Amount of interaction with attributes based on the interaction logs 

In [ ]:
## load all participants' interaction data with the UI
feedback_df = pd.read_csv(folder+'data/processed_data/Feedback_final.csv', delimiter=',', index_col=0)
feedback_df.drop(columns = ["Lower bound","Upper bound"],inplace = True) # blank columns
## get only feedback-related interactions with the UI
feedback_df = feedback_df[feedback_df["Function"] == "select_reject_application"] 
## load application data shown to participants through the UI
applications_df = pd.read_csv("Applications_Converted.csv", index_col=0)
## get list of weights before feedback integration in right order
features_in_trainingOrder = ['Number of children', 'Income', 'Loan Credit amount ', 'Loan annuity',
       'Goods Price', "Region's Normalized Population", 'Age',
       'Years in current employment', 'Years since changing registration', 'Years since changing  identity document',
       'Has Mobile', 'Has Employee Phone', 'Has Work Phone', 'Mobile reachable',
       'Has Phone', 'Has email', 'Number of family members', "Region's Rating",
       'Region & City Rating', 'Application Hour',
       'Contact address located in Registration region', 'Work address located in Registration region',
       'Contact address located in work region', 'Contact city in Registration city',
       'Work address in Registration city', 'Contact city in work city',
       'Number of times social circle at default risk for 30 days', 'Number of times social circle defaulted on loan for 30 days',
       'Number of times social circle were at risk to default on loan for 60 days', 'Number of times social circle defaulted on loan for 60 days',
       'Years since changing phone', 'Number of Credit Bureau enquiries 1 hour before application',
       'Number of Credit Bureau enquiries 1 day before application', 'Number of Credit Bureau enquiries 1 week before application',
       'Number of Credit Bureau enquiries 1 month before application', 'Number of Credit Bureau enquiries 3 month before application',
       'Number of Credit Bureau enquiries 1 year before application', 'Installments', 'Gender',
       'Owns Car', 'Owns Property', 'Accompanied while applying',
       'Income type', 'Highest education level',
       'Family status', 'Housing situation', 'Occupation Type',
       'Application Day', 'Employer organization']
feature_weights_dict = {}
for i in range(49):
    feature_weights_dict[applications_df['ft_name_'+str(i)].tolist()[0]] = applications_df['ft_weight_'+str(i)].tolist()[0]
feature_weights = []
for feat in features_in_trainingOrder:
    feature_weights.append(feature_weights_dict[feat])

## convert string values to list of dicts
for i in feedback_df.index :
    s = feedback_df['Value'].loc[i]
    # print(i,feedback_df['App ID'].loc[i])
    s = s.replace("'", "\"")
    s = s.replace("Region\"s Rating", "Region\'s Rating")
    s = s.replace("Region\"s Normalized Population", "Region\'s Normalized Population")
    feedback_df.loc[[i], "Value"] = pd.Series([json.loads(s)], index=[i])

## keep only feature weights relevant to each APP ID
all_new_values = []
for i in feedback_df.index :
    app_id = feedback_df['App ID'].loc[i]
    value = feedback_df['Value'].loc[i]
    new_value = {}
    if value:
        for v_dict in value:
            _idx = v_dict['attribute'].index('_')
            if app_id == np.int64(v_dict['attribute'][0:_idx]):
                attr = v_dict['attribute'][(_idx+1):]
                new_value[attr]=np.float64(v_dict['value'])
    all_new_values.append(new_value)    
feedback_df['Value'] = all_new_values
feedback_df

In [ ]:
attributes = {'Gender':'CODE_GENDER',
              'Family status':'NAME_FAMILY_STATUS',
              'Age':'AGE',
              "Region's Rating":'REGION_RATING_CLIENT',
              'Region & City Rating':'REGION_RATING_CLIENT_W_CITY',
              'Number of times social circle at default risk for 30 days':'OBS_30_CNT_SOCIAL_CIRCLE',
             'Number of times social circle defaulted on loan for 30 days':'DEF_30_CNT_SOCIAL_CIRCLE',
             'Number of times social circle were at risk to default on loan for 60 days':'OBS_60_CNT_SOCIAL_CIRCLE',
             'Number of times social circle defaulted on loan for 60 days':'DEF_60_CNT_SOCIAL_CIRCLE',
             'Number of children':'CNT_CHILDREN', 
              'Income':'AMT_INCOME_TOTAL', 
              'Loan Credit amount ':'AMT_CREDIT', 
              'Loan annuity':'AMT_ANNUITY',
               'Goods Price':'AMT_GOODS_PRICE', 
              "Region's Normalized Population":'REGION_POPULATION_RELATIVE',
               'Years in current employment':'YEARS_EMPLOYED', 
              'Years since changing registration':'YEARS_REGISTRATION', 
              'Years since changing  identity document':'YEARS_ID_PUBLISH',
               'Has Mobile':'FLAG_MOBIL', 
              'Has Employee Phone':'FLAG_EMP_PHONE', 
              'Has Work Phone':'FLAG_WORK_PHONE', 
              'Mobile reachable':'FLAG_CONT_MOBILE',
               'Has Phone':'FLAG_PHONE', 
              'Has email':'FLAG_EMAIL', 
              'Number of family members':'CNT_FAM_MEMBERS', 
              'Application Hour':'HOUR_APPR_PROCESS_START',
       'Contact address located in Registration region':'REG_REGION_NOT_LIVE_REGION', 
              'Work address located in Registration region':'REG_REGION_NOT_WORK_REGION',
       'Contact address located in work region':'LIVE_REGION_NOT_WORK_REGION', 
              'Contact city in Registration city':'REG_CITY_NOT_LIVE_CITY',
       'Work address in Registration city':'REG_CITY_NOT_WORK_CITY', 
              'Contact city in work city':'LIVE_CITY_NOT_WORK_CITY',
       'Years since changing phone':'YEARS_LAST_PHONE_CHANGE', 
              'Number of Credit Bureau enquiries 1 hour before application':'AMT_REQ_CREDIT_BUREAU_HOUR',
       'Number of Credit Bureau enquiries 1 day before application':'AMT_REQ_CREDIT_BUREAU_DAY', 
              'Number of Credit Bureau enquiries 1 week before application':'AMT_REQ_CREDIT_BUREAU_WEEK',
       'Number of Credit Bureau enquiries 1 month before application':'AMT_REQ_CREDIT_BUREAU_MON', 
              'Number of Credit Bureau enquiries 3 month before application':'AMT_REQ_CREDIT_BUREAU_QRT',
       'Number of Credit Bureau enquiries 1 year before application':'AMT_REQ_CREDIT_BUREAU_YEAR', 
              'Installments':'NAME_CONTRACT_TYPE', 
       'Owns Car':'FLAG_OWN_CAR', 
              'Owns Property':'FLAG_OWN_REALTY', 
              'Accompanied while applying':'NAME_TYPE_SUITE',
       'Income type':'NAME_INCOME_TYPE', 
              'Highest education level':'NAME_EDUCATION_TYPE',
              'Housing situation':'NAME_HOUSING_TYPE', 
              'Occupation Type':'OCCUPATION_TYPE ',
       'Application Day':'WEEKDAY_APPR_PROCESS_START', 
              'Employer organization':'ORGANIZATION_TYPE'}
attrs_count_dict = {}
parts_count = []
for attr in attributes:
    count = 0
    parts = []
    for part in feedback_df['ID'].unique():
        for idx in feedback_df[feedback_df['ID']==part].index:
            if attr in feedback_df.loc[idx]['Value']:
                if attributes[attr] not in attrs_count_dict:
                    attrs_count_dict[attributes[attr]] = 0
                count = count +1
                parts.append(part)
                break
    attrs_count_dict[attributes[attr]] = attrs_count_dict[attributes[attr]]+count
    parts_count.append(parts)
attrs_count_dict=sorted(attrs_count_dict.items(), key=lambda x:x[1], reverse=True)
print(attrs_count_dict)

In [ ]:
feature_weights_dict = {}
for i in range(49):
    feature_weights_dict[applications_df['ft_name_'+str(i)].tolist()[0]] = applications_df['ft_weight_'+str(i)].tolist()[0]
    
av_weight_ch_dict = {}
for attr in attributes:
    av_weight_ch_dict[attributes[attr]] = []
for i in feedback_df.index:
    app_id = feedback_df['App ID'].loc[i]
    value_dict = feedback_df['Value'].loc[i]
    if value_dict:
        for attr in value_dict:
            av_weight_ch_dict[attributes[attr]].append(value_dict[attr] - feature_weights_dict[attr])
for attr in av_weight_ch_dict:
    av_weight_ch_dict[attr] = sum(av_weight_ch_dict[attr])/len(av_weight_ch_dict[attr])
    idx = list(attributes.values()).index(attr)
    av_weight_ch_dict[attr] = (av_weight_ch_dict[attr]/abs(feature_weights[idx]))*100
av_weight_ch_dict = sorted(av_weight_ch_dict.items(), key=lambda x:x[1], reverse=True)
av_weight_ch_dict